In [251]:
import numpy as np
import json
import os
import pandas as pd

In [252]:
#Toma los archivos .json del directorio y los une a una base de datos usando pandas

data_list = []
base_dir = 'tech---ops/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df = pd.concat(data_list)


In [257]:
# Se agregaron las columnas nuevas segun las reacciones y ademas se ordenaron.

nuevas_columnas_reacciones = ['t_finalizado', 't_revisado', 't_resuelto']

df_columnas_reaccion = df.reindex(columns=df.columns.tolist() + nuevas_columnas_reacciones)
df_columnas_reaccion['reactions'].fillna('', inplace=True)
df_columnas_reaccion = df_columnas_reaccion.reset_index(drop=True)


In [260]:
df_columnas_reaccion['reactions']

0                                                       
1                                                       
2      [{'name': 'hot_pepper', 'users': ['U02TWSY0V3M...
3                                                       
4                                                       
                             ...                        
423                                                     
424                                                     
425    [{'name': 'scream', 'users': ['U02PKM216TX'], ...
426                                                     
427                                                     
Name: reactions, Length: 428, dtype: object

In [ ]:
#Tratando de iterar la columna reactions para poder almacenar True's & False's en las respectivas columnas segun la reaccion al thread.

simbolos_reaccion = ['white_check_mark', 'eyes', 'raised_hands']

for i in df_columnas_reaccion.index:
    if df_columnas_reaccion['reactions'][i] != '':
        for j in range(len(df_columnas_reaccion['reactions'][i])):
            for k in range(len(simbolos_reaccion)):
                if simbolos_reaccion[k] in df_columnas_reaccion['reactions'][i][j]:
                    df_columnas_reaccion[df_columnas_reaccion.columns[31 + i]][j] = True
                else:
                    df_columnas_reaccion[df_columnas_reaccion.columns[31 + i]][j] = False
                

In [202]:
# Exportar a excel

df_columnas_reaccion.to_excel('testslack.xlsx')

In [ ]:
# Seleccionamos las columnas utiles del df y ademas se elimnaron las filas duplicadas y se ordenaron segun la fecha y hora de la creacion del thread. Antes de hacer este paso, debo sacar la info de la columna reactions 

df_clean = df[['client_msg_id', 'text', 'user', 'ts', 'thread_ts', 'latest_reply']].drop_duplicates().sort_values(['thread_ts']).reset_index(drop=True)
df_clean.head()

In [ ]:
# Columnas nuevas a agregar

nuevas_columnas = ['nuevo_req', 'ticket_ops', 'ticket_tech', 'urgente', 't_finalizado', 't_revisado', 't_resuelto']

df_clean_new_columns = df_clean.reindex(columns=df_clean.columns.tolist() + nuevas_columnas)
df_clean_new_columns['latest_reply'].fillna('', inplace=True)



In [124]:
# Se creo una lista con los simbolos asociado a las nuevas columnas.

simbolos = [':writing_hand:', ':dart:', ':robot_face:',':rotating_light:']

# Se itero el df para asociarle un valor de True or False a las respectivas columnas segun la simbologia del mensaje principal del thread.

for i in df_clean_new_columns.index:
    if df_clean_new_columns['latest_reply'][i] != '':
            for j in range(len(simbolos)):
                if simbolos[j] in df_clean_new_columns['text'][i]:
                    df_clean_new_columns[df_clean_new_columns.columns[6 + j]][i] = True
                else:
                    df_clean_new_columns[df_clean_new_columns.columns[6 + j]][i] = False 

In [129]:
# Exportando data a excel

df_clean_new_columns.to_excel('data_slack.xlsx')